# 12. 图神经网络

安装`torch_geometric`, 依次在命令行执行:
```bash
> pip install torch-cluster
> pip install torch-spline-conv
> pip install torch-sparse
> pip install torch-scatter
> pip install torch-geometric
```

- 导入`torch_geometric`

In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_networkx
from torch_geometric.data import Data

- 构建GNN

Its node-wise formulation is given by:
$$
        \mathbf{x}^{\prime}_i = \mathbf{\Theta}^{\top} \sum_{j \in
        \mathcal{N}(v) \cup \{ i \}} \frac{e_{j,i}}{\sqrt{\hat{d}_j
        \hat{d}_i}} \mathbf{x}_j
$$

with $\hat{d}_i = 1 + \sum_{j \in \mathcal{N}(i)} e_{j,i}$, where
$e_{j,i}$ denotes the edge weight from source node :obj:`j` to target
node :obj:`i` (default: :obj:`1.0`)

In [2]:
class GCN(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, 16)  # 第一层图卷积
        self.conv2 = GCNConv(16, num_classes)  # 第二层图卷积

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [3]:
dataset = TUDataset(root='dataset/', name='ENZYMES')

In [4]:
dataset = dataset.shuffle()
len(dataset)

600

In [5]:
dataset.num_classes, dataset.num_node_features

(6, 3)

In [6]:
data = dataset[0]

In [10]:
dataset[3]

Data(edge_index=[2, 70], x=[18, 3], y=[1])

In [70]:
data.is_undirected()

True

In [71]:
train_dataset = dataset[:540]

In [79]:
train_dataset

ENZYMES(540)

In [72]:
test_dataset = dataset[540:]

In [84]:
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [87]:
for batch in loader:
    batch
    break

In [89]:
batch.num_graphs

32

In [95]:
len(batch.edge_index[0])

4040

In [93]:
len(batch.x)

1091

In [62]:
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
data = Data(x=x, edge_index=edge_index)

In [97]:
model = GCN(dataset.num_features, dataset.num_classes)
criterion = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [99]:
def train(model, optimizer, data):
    model.train()
    optimizer.zero_grad()
    output = model(data.x, data.edge_index)
    loss = criterion(output[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [ ]:
for epoch in range(200):
    train(model, optimizer, data)